## Importación de módulos

In [31]:
import cv2
import mediapipe as mp
import os
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

## Aplicación interactiva

In [33]:
# Definir funciones y opciones
BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
GestureRecognizerResult = mp.tasks.vision.GestureRecognizerResult
VisionRunningMode = mp.tasks.vision.RunningMode

model_path = os.path.abspath('gesture_recognizer.task')
print("Ruta del modelo:", model_path)

def print_result(result: GestureRecognizerResult, output_image: mp.Image, timestamp_ms: int):
    print('gesture recognition result: {}'.format(result))

# Crear opciones para el reconocedor de gestos
options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result)

# Crear el reconocedor de gestos
with GestureRecognizer.create_from_options(options) as recognizer:
    # Inicializar la captura de video desde la cámara
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        # Leer un frame de la cámara
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir el frame a RGB (MediaPipe requiere imágenes en formato RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Convertir el frame de OpenCV a un objeto Image de MediaPipe
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

        # Obtener la marca de tiempo del frame
        frame_timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

        # Realizar el reconocimiento de gestos en el frame actual
        recognizer.recognize_async(mp_image, frame_timestamp_ms)

        # Hacer algo con el frame si es necesario
        # Por ejemplo, dibujar puntos, líneas, etc.

        # Mostrar la imagen resultante
        cv2.imshow('Gesture Recognition', frame)

        # Salir del bucle si se presiona 'Esc'
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

    

RuntimeError: Unable to open file at c:\Anaconda\envs\mediapipe\Lib\site-packages/f:\ULPGC\Cuarto\VC\Practica7\VC_P7\gesture_recognizer.task, errno=22